In [5]:
!pip install datasets

In [6]:
#здесь производятся все импорты библиотек, использующихся в данном решении
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
import os
import gc
import scipy.io as sio
import time
import os
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
torch.manual_seed(42)
import re
from transformers import logging
logging.set_verbosity_error()
from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef
from torchvision import datasets, transforms as T
from transformers import AutoModel, BertModel
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification
from scipy.spatial.distance import cosine
import warnings
from sklearn import metrics
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [8]:
#чтение данных и их просмотр
df = pd.read_csv("data/bazaforclassify.csv")
df['Категория'].value_counts()



,count
Категория,
Забота о работниках,138
Забота компании о работнике,78
Заработная плата,67
Организация работы,65
Оплата труда,59
ЗОЖ,49
Жилищная политика,13
Коллективный договор,12
Режим труда и отдыха,12


In [10]:
#label_encoding и препроцессинг для лучшего результата модели
from sklearn import preprocessing

rev = {}
label_encoder = preprocessing.LabelEncoder()
df['target'] = label_encoder.fit_transform(df['Категория'])
enc = df['target'].unique().tolist()
base = df['Категория'].unique().tolist()
for i in range(len(enc)):
  rev[enc[i]] = base[i]
df

,Категория,Дополнительные теги,Вопрос,Краткий ответ,Ссылка на развернутый ответ,Развернутый ответ,target
0,Забота о работниках,детский сад,Размер оплаты за посещение дошкольного учрежде...,Оплата за посещение ребёнком детского сада опр...,NaN,NaN,4
1,Забота о работниках,детский сад,Сколько мне будет необходимо платить за корпор...,Оплата за посещение ребёнком детского сада опр...,NaN,NaN,4
2,Забота о работниках,детский сад,сколько платить за ребенка в корпоративном дет...,Оплата за посещение ребёнком детского сада опр...,NaN,NaN,4
3,Забота о работниках,детский сад,Оплата за детский сад,Оплата за посещение ребёнком детского сада опр...,NaN,NaN,4
4,Забота о работниках,детский сад,сколько платить за ребенка в детском саду,Оплата за посещение ребёнком детского сада опр...,NaN,NaN,4
...,...,...,...,...,...,...,...
507,Оплата труда,Командировочные и суточные,Сколько заплатят за работу в командировке помо...,Согласно Положению о порядке и размерах возмещ...,NaN,NaN,8
508,Оплата труда,Командировочные и суточные,размер командировочных для дирекции тяги,Согласно Положению о порядке и размерах возмещ...,NaN,NaN,8
509,Оплата труда,Командировочные и суточные,суточные для работников локомотивного комплекса,Согласно Положению о порядке и размерах возмещ...,NaN,NaN,8
510,Оплата труда,Командировочные и суточные,как оплачивают командировку машинистам,Согласно Положению о порядке и размерах возмещ...,NaN,NaN,8


In [11]:
#разделение на валидационную и обучающую выборку
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['target'], shuffle=True)

In [12]:
#установка сида для модели для большей предсказуемости ее обучения
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False
seed_all(42)

In [13]:
#Создание структур и датасетов для понимания данных нейросетью
class ClassificationDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.text = data['Вопрос'].to_list()
        self.target = data['target'].to_list()

    def __getitem__(self, idx):
        text, target = self.text[idx],self.target[idx]
        return text, target

    def __len__(self):
        return len(self.text)

def collate_fn(batch):
    model_input = []
    model_target = []
    for text, target in batch:
        model_input.append(text)
        model_target.append(target)

    tok = tokenizer(model_input, padding=True,
                    max_length=512, truncation=True,
                    return_tensors='pt')
    return tok, torch.tensor(model_target).long()


def get_loader(dataset, shuffle, batch_size):
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=False,
        num_workers=0,
        collate_fn=collate_fn
    )
    return loader

In [14]:
#Архитектура нейросети
class BertCLS(nn.Module):
    def __init__(self, model, n_classes):
        super(BertCLS, self).__init__()
        self.model = model
        self.fc = nn.Linear(1024, n_classes)

    def forward(self, batch):
        return self.fc(self.model(**batch).pooler_output)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "deepvk/USER-bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)


model = BertModel.from_pretrained(
            model_name,
            ignore_mismatched_sizes=True,
            num_labels=12
        )
bert_cls = BertCLS(model, n_classes=12)

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/697 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

In [15]:
def clear_cache():
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    torch.backends.cudnn.deterministic=True

In [16]:
#параметры обучения
set_seed(42)
bert_cls = bert_cls.to(device)

num_epochs = 10
gradient_accumulation_steps = 1

batch_size = 32
train_dataset = ClassificationDataset(X_train)
train_loader = get_loader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = ClassificationDataset(X_test)
test_loader = get_loader(test_dataset, shuffle=False, batch_size=batch_size)

# Total training steps calculation
total_steps = len(train_loader) * num_epochs // gradient_accumulation_steps

# Optimizer with a smaller learning rate and weight decay for regularization
optimizer = optim.AdamW(bert_cls.parameters(), lr=3e-5, weight_decay=0.01)

scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-5,
                                          total_steps=total_steps,
                                          div_factor=25,
                                          pct_start=0.1)
loss_func = torch.nn.CrossEntropyLoss()

In [18]:
#Функции обучения, предикта и валидации модели
def train(model, loss_func, device, train_loader, optimizer, epoch, gradient_accumulation_steps, scheduler):
    model.train()
    train_loader_length = len(train_loader) - 1 # хотим знать когда эпоха закончится
    pbar = tqdm(train_loader)
    for batch_idx, (data, labels) in enumerate(pbar):
        data, labels = data.to(device), labels.to(device)

        t = time.time()
        embeddings = model(data)
        t2 = time.time() - t

        loss = loss_func(embeddings, labels) / gradient_accumulation_steps

        pbar.set_description(f"Loss Train: {float(loss) * gradient_accumulation_steps} Time: {t2}")

        loss.backward()

        if (batch_idx % gradient_accumulation_steps == 0) or (batch_idx == train_loader_length):
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1) # ставим модель в рамки, помогает
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

        del data, labels, embeddings, loss
    return model


def test(model, loader, device):
    y_true = []
    pred = []
    model.eval()
    with torch.no_grad():
        pbar = tqdm(loader)
        for batch_idx, (data, labels) in enumerate(pbar):
            data = data.to(device)
            embeddings = model(data)
            pred.extend(embeddings.argmax(-1).detach().cpu().numpy())
            y_true.extend(labels.detach().cpu().numpy())

    return y_true, pred

def metrics(y_true, pred):
    print("matthews_corrcoef:", matthews_corrcoef(y_true, pred))
    print("accuracy_score:", accuracy_score(y_true, pred))
    print("f1_score:", f1_score(y_true, pred, average='weighted'))

In [18]:
#Обучение нейросети

for x in bert_cls.parameters(): x.requires_grad = True
for epoch in range(1, num_epochs + 1):
    bert_cls.train()

    clear_cache()
    train(bert_cls, loss_func, device, train_loader, optimizer,
          epoch, gradient_accumulation_steps, scheduler)
    PATH = f"./models-all-classes/{bert_cls.__class__.__name__}_epoch_{epoch}.pth"


    clear_cache()
    y_true, pred = test(bert_cls, test_loader, device)
    metrics(y_true, pred)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

matthews_corrcoef: 0.4033935461849601
accuracy_score: 0.4807692307692308
f1_score: 0.3514328808446455


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

matthews_corrcoef: 0.9323734729493892
accuracy_score: 0.9423076923076923
f1_score: 0.9437869822485206


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

matthews_corrcoef: 0.9323734729493892
accuracy_score: 0.9423076923076923
f1_score: 0.9437869822485206


In [19]:
#Сохранение нейросети
torch.save(bert_cls.state_dict(), 'model')

In [19]:
#Тестирование
def test(model, loader, device):
    y_true = []
    pred = []
    model.eval()
    with torch.no_grad():
        pbar = tqdm(loader)
        for batch_idx, (data, labels) in enumerate(pbar):
            data = data.to(device)
            embeddings = model(data)
            pred.extend(embeddings.argmax(-1).detach().cpu().numpy())
            y_true.extend(labels.detach().cpu().numpy())

    return y_true, pred
test_dataset1 = ClassificationDataset(X_test)
test_loader1 = get_loader(test_dataset1, shuffle=False, batch_size=1)
y_true, pred = test(bert_cls,test_loader1,'cuda')
metrics(y_true, pred)

100%|██████████| 52/52 [00:01<00:00, 39.57it/s]

matthews_corrcoef: 0.9323734729493892
accuracy_score: 0.9423076923076923
f1_score: 0.9437869822485206


In [4]:
#Код для использования нейросети(Функция предикт принимает вопрос и выдает наиболее близкий к нему класс)
import pandas as pd
import numpy as np
import random
import torch
import transformers
import torch.nn as nn
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertModel
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
rev = {4: 'Забота о работниках', 3: 'Забота компании о работнике', 2: 'ЗОЖ', 1: 'Жилищная политика', 6: 'Коллективный договор', 11: 'Режим труда и отдыха', 7: 'Молодежная политика', 0: 'Адаптация', 5: 'Заработная плата', 8: 'Оплата труда', 10: 'Организация труда', 9: 'Организация работы'}
class BertCLS(nn.Module):
    def __init__(self, model, n_classes):
        super(BertCLS, self).__init__()
        self.model = model
        self.fc = nn.Linear(1024, n_classes)

    def forward(self, batch):
        return self.fc(self.model(**batch).pooler_output)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "deepvk/USER-bge-m3"
tokenizer = AutoTokenizer.from_pretrained(model_name)


model = BertModel.from_pretrained(
            model_name,
            ignore_mismatched_sizes=True,
            num_labels=12
        )
bert_cls = BertCLS(model, n_classes=12).to(device)
bert_cls.load_state_dict(torch.load("model", weights_only=True))
bert_cls = bert_cls.to(device)
class ClassificationDataset_test(Dataset):
    def __init__(self, data):
        super().__init__()
        self.text = data

    def __getitem__(self, idx):
        text = self.text[idx]
        return text

    def __len__(self):
        return len(self.text)

def collate_fn(batch):
    model_input = []
    for text in batch:
        model_input.append(text)

    tok = tokenizer(model_input, padding=True,
                    max_length=512, truncation=True,
                    return_tensors='pt')
    return tok


def get_loader(dataset, shuffle, batch_size):
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=False,
        num_workers=0,
        collate_fn=collate_fn
    )
    return loader
def ans(model, loader, device):
    pred = []
    model.eval()
    with torch.no_grad():
        pbar = tqdm(loader)
        for batch_idx, data in enumerate(pbar):
            data = data.to(device)
            embeddings = model(data)
            pred.extend(embeddings.argmax(-1).detach().cpu().numpy())

    return pred

def predict(s):
  global rev, bert_cls, device
  test_dtst = ClassificationDataset_test([s])
  test_loader1 = get_loader(test_dtst, shuffle=False, batch_size=1)
  preds  = ans(bert_cls, test_loader1, device)
  fin = []
  for i in preds:
    fin.append(rev[i])
  return fin[0]

#Пример использования
predict("Что такое коллективный договор?")


You are using a model of type xlm-roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
100%|██████████| 1/1 [00:00<00:00,  1.05it/s]


'Коллективный договор'

In [ ]:
|